In [1]:
from init import *

code = r'''
#include <eosio/crypto.hpp>
#include <eosio/print.hpp>
#include <eosio/name.hpp>

using namespace eosio;

#define N(name) name##_n.value

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        checksum160 hash160 = sha1("hello,worldd", 12);
        printhex((void *)hash160.data(), 20);print("\n");

        assert_sha1("hello,worldd", 12, hash160);

        checksum256 hash256 = sha256("hello,world", 12);
        printhex((void *)hash256.data(), 32);print("\n");
        assert_sha256("hello,world", 12, hash256);

        checksum512 hash512 = sha512("hello,world", 12);
        printhex((void *)hash512.data(), 64);print("\n");
        assert_sha512("hello,world", 12, hash512);

        checksum160 hash = ripemd160("hello,world", 12);
        printhex((void *)hash.data(), 20);print("\n");
        assert_ripemd160("hello,world", 12, hash);
    }
}
'''

publish_cpp_contract('helloworld11', code)
print('done!')



update contract
done!


In [40]:
name = 'helloworld11'
try:
    r = eosapi.push_action(name, 'sayhello', b'hello,world', {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

dccf48347b12434e0b8568d6b7de7d4000000000
5d78ff1f609563113144c53353b5c67dca8a6203b8b1ba8c33a58c7858cef3aa
9124d391e6ea07442f378898c85a2521034ed16c17378ff6ed3fa37b0dea8b12dee106e6d87b0d5d74685d7cfbc7b0ecdcba0c9ed3126fd2e2591a174d343cb4
9fbcc36bef806c7caaec414590db060900000000



In [17]:
from init import *

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"text", "type":"bytes"},
        {"name":"sign", "type":"signature"}
        ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

code = r'''
#include <vector>
#include <eosio/action.hpp>
#include <eosio/crypto.hpp>
#include <eosio/print.hpp>
#include <eosio/name.hpp>

using namespace std;
using namespace eosio;

#define N(name) name##_n.value

struct mystruct {
    vector<char> text;
    signature sign;
    EOSLIB_SERIALIZE( mystruct, (text)(sign) )
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        mystruct s = unpack_action_data<mystruct>();
        checksum256 digest = sha256(s.text.data(), s.text.size());
        public_key key = recover_key(digest, s.sign);
        printhex((void *)key.data.data(), key.data.size());
        assert_recover_key(digest, s.sign, key);
    }
}
'''

publish_cpp_contract('helloworld11', code, abi)
print('done!')



update contract
done!


In [18]:
name = 'helloworld11'
try:
    text = b'hello,world'
    import hashlib
    h = hashlib.sha256()
    h.update(text)
    sign = wallet.sign_digest(h.digest(), 'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV')
    print(sign)
#    args = {'text':text.hex(), 'sign': sign}
    args = {'text':text.hex()}
    args = {'sign': sign}
    args = {'text':text.hex(), 'sign': sign}
    r = eosapi.push_action(name, 'sayhello', args, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
except Exception as e:
    print(e)

SIG_K1_JzwaU4Zf1ZzyRDsg5EtoThjqtECKMcKozDTHwEtBpqoCQAYu1JnZXSTc7Tjpb1AzLNxu8gCXRrHYhLgz7v9M2iu3Th8Qow
036bfc5cc1e9215625a2f25fd1013cde6e7b9d6f294753cce498f575a2e734a700


In [19]:
import base58

public_key = bytes.fromhex('036bfc5cc1e9215625a2f25fd1013cde6e7b9d6f294753cce498f575a2e734a700')
print(public_key)

h = hashlib.new('ripemd160')
h.update(public_key)
digest = h.digest()
public_key += digest[:4]
b'EOS'+base58.b58encode(public_key)

b'\x03k\xfc\\\xc1\xe9!V%\xa2\xf2_\xd1\x01<\xden{\x9do)GS\xcc\xe4\x98\xf5u\xa2\xe74\xa7\x00'


b'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'